In [21]:
import os
import re
import pickle
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Embedding, Bidirectional, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
# from keras.layers import Embedding
from  keras . utils  import  to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Permute
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Embedding

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd 
total_data=pd.read_csv('gdrive/My Drive/Thesis Data/wiki doc csv/wiki.csv')


In [4]:
!git clone -l -s https://github.com/banglakit/bengali-stemmer.git

Cloning into 'bengali-stemmer'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 94 (delta 5), reused 16 (delta 4), pack-reused 69
Unpacking objects: 100% (94/94), done.


In [5]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git    

  Cloning https://github.com/banglakit/bengali-stemmer.git to /tmp/pip-req-build-ujowp87c
  Running command git clone -q https://github.com/banglakit/bengali-stemmer.git /tmp/pip-req-build-ujowp87c
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6393 sha256=1b8168a1d6024763335dfe24394658aa038fa1f0dc2ecbc78c2d0a6565f3f5d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-hovnimv6/wheels/a1/ad/a1/4ba354059b17c00600a14e13a504e7bdb49f20f2f4e2f3639c
Successfully built bengali-stemmer


In [6]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git    

  Cloning https://github.com/banglakit/bengali-stemmer.git to /tmp/pip-req-build-v483gsmh
  Running command git clone -q https://github.com/banglakit/bengali-stemmer.git /tmp/pip-req-build-v483gsmh
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6393 sha256=38a51e5e392c7227522840b6b4a1788088745c07da7b39a2c9f89bd9bcb4d61b
  Stored in directory: /tmp/pip-ephem-wheel-cache-qw00915c/wheels/a1/ad/a1/4ba354059b17c00600a14e13a504e7bdb49f20f2f4e2f3639c
Successfully built bengali-stemmer


In [7]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()
stemmer.stem_word('বাংলায়')

'বাংলা'

In [8]:
macronum=sorted(set(total_data['title']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

total_data['title']=total_data['title'].apply(fun)


In [9]:
texts = list(total_data['text'])
labels = list(total_data['title'])

In [10]:
total_data.head()

,id,text,title,url
0,2341,বংশী বাংলাদেশের একটি ক্ষুদ্রতম উপজাতি। এরা টাঙ...,38938,https://bn.wikipedia.org/wiki?curid=2341
1,107282,আবহাওয়াবিদ্যা (বা আবহবিদ্যা) মানে এককথায় আবহ...,4576,https://bn.wikipedia.org/wiki?curid=107282
2,107838,কোরিয়ার ওয়ার্কার্স পার্টি হল গণতান্ত্রিক গণপ...,16104,https://bn.wikipedia.org/wiki?curid=107838
3,4542,মোহাম্মদ মোস্তফা কামাল (১৬ ডিসেম্বর ১৯৪৭ - এপ্...,51128,https://bn.wikipedia.org/wiki?curid=4542
4,4626,মণিপুরী সংস্কৃতির উজ্জ্বলতম দিক হলো মণিপুরী নৃ...,46438,https://bn.wikipedia.org/wiki?curid=4626


In [16]:
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned)
    return cleaned1

In [17]:
stopwords = pd.read_csv('gdrive/My Drive/Colab Notebooks/Stopwords.csv')

In [18]:
set_stop = set(stopwords['words'])

In [19]:

def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []

        for w in sentence.split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                    if((cleaned_word) not in set_stop):
                        s = stemmer.stem_word(cleaned_word)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string

In [22]:
texts = pre_process(texts)

In [23]:
texts[0]

'বংশী বাংলাদেশ ক্ষুদ্রতম উপজাতি টাঙ্গাইল জেল মহানান্দপুর দন্দোনিয়া নাম পাশাপাশি দুই গ্রাম বসবাস নিজে সূর্য বংশী'